# XGBoost

## 1. Setup and Imports

In [ ]:
import os
import sys

try:
    base_path = os.path.dirname(__file__)
except NameError:
    base_path = os.getcwd()

# 1️⃣ Ajouter le dossier parent (un cran au-dessus)
parent_dir = os.path.abspath(os.path.join(base_path, '..'))
sys.path.append(parent_dir)

# 2️⃣ Ajouter le dossier parent du dossier parent (deux crans au-dessus)
two_up_dir = os.path.abspath(os.path.join(base_path, '..', '..'))
sys.path.append(two_up_dir)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Import the environment and agents
from permuted_mnist.env.permuted_mnist import PermutedMNISTEnv
from models.Xg_boost.xg_boost import Agent as Xg_boost_Agent

print("✓ Imports successful")

✓ Imports successful


We fix the seed for reproducibility:

In [3]:
seed = 42

We fix the number of CPUs:

In [4]:
n_jobs = 2

## 2. Create the Environment

Let's create an environment with 10 different permuted tasks:

In [5]:
# Create environment with 10 episodes (tasks)
env = PermutedMNISTEnv(number_episodes=10)

# Set seed for reproducibility
env.set_seed(seed)

print(f"Environment created with {env.number_episodes} permuted tasks")
print(f"Training set size: {env.train_size} samples")
print(f"Test set size: {env.test_size} samples")

Environment created with 10 permuted tasks
Training set size: 60000 samples
Test set size: 10000 samples


## XGBoost :

In [ ]:
def xg_boost(n_est, max_depth, lr):
    # Reset environment for fresh start
    env.reset()
    env.set_seed(seed)

    # Create Xg_boost agent
    xg_boost_agent = Xg_boost_Agent(n_est, max_depth, lr, n_jobs, seed)

    # Track performance
    xg_boost_accuracies = []
    xg_boost_times = []

    print(f"Evaluating Xg_boost Agent with n_estimators={n_est}, max_depth={max_depth} and learning_rate={lr}:")

    # Evaluate on all tasks
    task_num = 1
    while True:
        task = env.get_next_task()
        if task is None:
            break
        if task_num >= 4: # limited for quick test
            break
            
        # Reset agent for new task
        xg_boost_agent.reset()
    
        start_time = time.time()
    
        # Train
        xg_boost_agent.train(task['X_train'], task['y_train'])
    
        # Make predictions
        predictions = xg_boost_agent.predict(task['X_test'])
    
        # Calculate time and accuracy
        elapsed_time = time.time() - start_time
        accuracy = env.evaluate(predictions, task['y_test'])
    
        xg_boost_accuracies.append(accuracy)
        xg_boost_times.append(elapsed_time)
    
        print(f"Task {task_num}: Accuracy = {accuracy:.2%}, Time = {elapsed_time:.4f}s")
        task_num += 1

    mean_accuracy = np.mean(xg_boost_accuracies)
    std_accuracy = np.std(xg_boost_accuracies)
    total_time = np.sum(xg_boost_times)

    print(f"\nXg_boost Agent Summary:")
    print(f"  Mean accuracy: {mean_accuracy:.2%} ± {std_accuracy:.2%}")
    print(f"  Total time: {total_time:.2f}s")

    return mean_accuracy

In [9]:
n_estimators = [25] 
max_depths = [5] 
learning_rates = [0.66] 
best_mean_accuracy = 0 

best_n_estimators = None 
best_max_depth = None 
best_lr = None 

for n_est in n_estimators: 
    for max_depth in max_depths: 
        for lr in learning_rates: 
            mean_accuracy = xg_boost(n_est, max_depth, lr) 
            print('-' * 50) 
            
            if mean_accuracy > best_mean_accuracy: 
                best_mean_accuracy = mean_accuracy 
                best_n_estimators = n_est 
                best_max_depth = max_depth 
                best_lr = lr 

print(f"The best number of estimators is: {best_n_estimators}") 
print(f"The best maximum depth is: {best_max_depth}") 
print(f"The best learning rate is: {best_lr}") 
print(f"The associated mean accuracy is: {best_mean_accuracy:0.4f}")

Evaluating Xg_boost Agent with n_estimators=25, max_depth=5 and learning_rate=0.66:
Task 1: Accuracy = 96.71%, Time = 70.5925s
Task 2: Accuracy = 96.75%, Time = 60.4895s
Task 3: Accuracy = 96.42%, Time = 54.8462s

Xg_boost Agent Summary:
  Mean accuracy: 96.63% ± 0.15%
  Total time: 185.93s
--------------------------------------------------
The best number of estimators is: 25
The best maximum depth is: 5
The best learning rate is: 0.66
The associated mean accuracy is: 0.9663


XGBoost a été choisi pour ce problème de classification sur MNIST perturbé en raison de son excellent compromis entre performance, flexibilité et efficacité sur CPU. Contrairement aux modèles linéaires comme la régression logistique, XGBoost capture des interactions non linéaires entre les pixels sans nécessiter de réseau de neurones, ce qui le rend adapté à des variations complexes comme les permutations de pixels. De plus, il est hautement optimisé pour le calcul parallèle, permettant d’exploiter efficacement les deux cœurs disponibles tout en respectant les contraintes de mémoire et de temps imposées par le challenge. Enfin, son contrôle précis des hyperparamètres (profondeur des arbres, taux d’apprentissage, nombre d’estimateurs, etc.) offre une bonne balance entre vitesse d’entraînement et capacité de généralisation, ce qui en fait un candidat pertinent dans un contexte de tâches successives et limitées en ressources.